In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus, get_hyphae
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from sample.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children, resolve_ambiguity, relabel_nodes_after_amb
from collections import Counter
from IPython.display import clear_output
from sample.notebooks.analysis.data_info import *
from sample.pipeline.functions.node_id import reconnect_degree_2
from sample.pipeline.functions.extract_graph import (
    generate_skeleton,
    from_nx_to_tab,
    prune_graph
)

from sample.pipeline.functions.hyphae_id_surf import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    clean_obvious_fake_tips,
    width_based_cleaning
)
from sample.pipeline.functions.experiment_class_surf import clean_exp_with_hyphaes
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from scipy.optimize import curve_fit
from shapely.geometry import Polygon, shape


In [38]:
directory = directory_project
plate_number = 102
plate = get_postion_number(plate_number)
begin = 262
end = 287
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp = Experiment(plate,directory)
exp.load(dates) #for method 2

In [51]:
directory = directory_project
plate_number = 59
plate = get_postion_number(plate_number)
begin = 7
end = 42
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp = Experiment(plate,directory)
exp.load(dates) #for method 2

In [2]:
directory = directory_project
plate_number = 94
plate = get_postion_number(plate_number)
begin = 269
end = 329
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp = Experiment(plate,directory)
exp.load(dates) #for method 2

In [93]:
directory = directory_project
plate_number = 69
plate = get_postion_number(plate_number)
begin = 153
end = 190
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp = Experiment(plate,directory)
exp.load(dates) #for method 2

In [3]:
lengths = [get_length(exp,t) for t in range(exp.ts)]
times = [get_time(exp,0,t) for t in range(exp.ts)]
lengths = np.array(lengths)
times = np.array(times)

In [7]:
areas = []   
for t in range(exp.ts):
    nodes = np.array([node.pos(t) for node in exp.nodes if node.is_in(t)])
    hull=scipy.spatial.ConvexHull(nodes)
    poly = Polygon([nodes[vertice] for vertice in hull.vertices])
    areas.append(poly.area* 1.725**2/(1000**2))


In [8]:
speeds = (lengths[1:]-lengths[:-1])/(times[1:]-times[:-1])

In [12]:
end = 30
begin = 0
lengths_select = lengths[begin:end]
times_select = times[begin:end]
areas_select = areas[begin:end]

In [13]:

coefs = np.polyfit(times_select,areas_select ,2)
p = np.poly1d(coefs)
def func(t,a,t0):
    return(a*(t-t0)**2)
popt0, pcov = curve_fit(func, times_select,areas_select ,bounds = ([0,-np.inf],2*[np.inf]),p0=[1,1])
def func2(t,alpha,C,d):
    return(C*(np.exp(alpha*(t)))+d)
popt1, pcov = curve_fit(func2, times_select, lengths_select/1000,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])


In [17]:
(2.84+1.10)*1000/np.log(2)/24

236.8424358792715

In [14]:
fig=plt.figure(figsize=(9,8))
ax = fig.add_subplot(111)
ax.scatter(times_select,areas_select ,label = 'data areas ($mm^2$)')
ax.plot(times_select,p(np.array(times_select)), label= f'quadratic fit a={"{:.2f}".format(coefs[0])} mm^2.h-2')
# ax.plot(times,func(np.array(times),*popt0),label = f'quadratic fit a={"{:.2f}".format(popt0[0])} mm^2.h-2')
ax.set_xlabel('time (h)')
ax.set_ylabel('network area $(mm^2)$')
plt.legend()
ax2 = ax.twinx()
ax2.scatter(times_select,lengths_select/1000,color='red', label = 'data length ($mm$)')
ax2.plot(times_select,func2(np.array(times_select),*popt1),label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='red')

ax2.set_ylabel('network length $(mm)$')

plt.legend(loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
np.sqrt(coefs[0]/(np.pi/2))*1000

318.39400055332436

In [7]:
end = 35
fig=plt.figure(figsize=(9,8))
ax = fig.add_subplot(111)
ax.scatter(times[:-1],speeds ,label = 'data areas ($mm^2$)')
def func3(t,alpha,C):
    return(C*(np.exp(alpha*(t))))
popt2, pcov = curve_fit(func3, times[:end], speeds[:end],bounds = ([0,0],2*[np.inf]),p0=[1,1])
ax.plot(times[:end],func3(np.array(times[:end]),*popt2),label = f'exponential fit : alpha= {"{:.2f}".format(popt2[0]*24)}.day-1',color='red')

plt.legend(loc = 'upper right')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
exp.load_compressed_skel()

In [36]:
def get_density_maps(exp,t,compress,kern_sizes):
    skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
    window=compress
    densities=np.zeros((skeletons[t].shape[0]//compress,skeletons[t].shape[1]//compress),dtype=np.float)
    for xx in range(skeletons[t].shape[0]//compress):
        for yy in range(skeletons[t].shape[1]//compress):
            x = xx*compress
            y = yy*compress
            skeleton=skeletons[t][x-window:x+window,y-window:y+window]
            density = skeleton.count_nonzero()/((window*1.725)**2)
            densities[xx,yy]=density
    results = {}
    for kern_size in kern_sizes:
        density_filtered = gaussian_filter(densities,kern_size)
        sx = ndimage.sobel(density_filtered ,axis=0,mode='constant')
        sy = ndimage.sobel(density_filtered ,axis=1,mode='constant')
        sobel=np.hypot(sx,sy)
        results[kern_size] = density_filtered,sx,sy,sobel
    return(results)

In [ ]:
kern_size=20
density_maps = [get_density_maps(exp,t,100,[kern_size]) for t in range(exp.ts)]

In [ ]:
for index,density_map in enumerate(density_maps):
    plt.close('all')
    clear_output(wait=True)
    fig=plt.figure(figsize=(14,12))
    ax = fig.add_subplot(111)
    im= density_map[kern_size][0]
    figure = ax.imshow(im>=0.001,vmax = 0.01)
    plt.colorbar(figure,orientation = 'horizontal')
    save = f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/im*{index}.png'
    plt.savefig(save)

In [ ]:
img_array = []
for index in range(len(density_maps)):
    img = cv2.imread(f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/im*{index}.png')
    img_array.append(img)
imageio.mimsave(f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/movie_dense_{kern_size}thresh.gif', img_array,duration = 1)

In [7]:
cmap=plt.get_cmap('inferno')

In [9]:
plt.close('all')
fig=plt.figure(figsize=(12,14))
ax = fig.add_subplot(111)
areas = []   
ax.set_xlim(6000*5)
ax.set_ylim(12000*5)
for t in range(exp.ts):
    nodes = np.array([node.pos(t) for node in exp.nodes if node.is_in(t)])
    hull=scipy.spatial.ConvexHull(nodes)
    for simplex in hull.simplices:
        ax.plot(nodes[simplex, 0], nodes[simplex, 1],color=cmap(t*3))
    poly = Polygon([nodes[vertice] for vertice in hull.vertices])
    areas.append(poly.area* 1.725**2/(1000**2))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig=plt.figure(figsize=(9,8))
ax = fig.add_subplot(111)
for simplex in hull.simplices:
    ax.plot(nodes[simplex, 0], nodes[simplex, 1], 'k-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[14  0]
[52  0]
[147  14]
[117  52]
[148 149]
[148 147]
[146 149]
[146 117]


In [4]:
sizes = [1,2,3]

In [14]:
sizes = np.load(f'/home/cbisot/pycode/MscThesis/Results/sizes_{plate_number}_{begin}_{end}.npy')
areas = sizes * 1.725**2/(1000**2)

In [12]:
speeds = (lengths[1:]-lengths[:-1])/(times[1:]-times[:-1])

In [56]:
end = 35
fig=plt.figure(figsize=(9,8))
ax = fig.add_subplot(111)
ax.scatter(times[:-1],speeds ,label = 'data areas ($mm^2$)')
def func3(t,alpha,C):
    return(C*(np.exp(alpha*(t))))
popt2, pcov = curve_fit(func3, times[:end], speeds[:end],bounds = ([0,0],2*[np.inf]),p0=[1,1])
ax.plot(times[:end],func3(np.array(times[:end]),*popt2),label = f'exponential fit : alpha= {"{:.2f}".format(popt2[0]*24)}.day-1',color='red')

plt.legend(loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
data = pd.DataFrame(list(zip(speeds,times[:-1])),columns = ['speed','time'])
# data['straight'] = data['straight_distance']*1.725/data['curv_growth']
# data['straight_scaled'] = data['straight_distance']*1.725/np.sqrt(data['curv_growth'])
# data['time'] = lapses_time
# data['straight_time'] = data['straight']/data['time']
# data['speed'] = data['curv_growth']/data['time']
correct = data.loc[data['speed']<500000]

In [8]:
plt.close('all')
abcisse = 'time'
ordinate = 'speed'
tab = correct
baits_sort = tab.sort_values(abcisse)
N=4
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')

ax.set_ylabel('Network scale growth speed ($\mu m .h^{-1}$)')
# ax.set_xlim(-0.5,0.5)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))

ax.scatter(tab[abcisse],tab[ordinate],alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [13]:
plt.close('all')
exp.plot_raw(20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
def func3(t):
    return(func2(t,*popt1)/p(t))

In [18]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 8))
ax.scatter(times,lengths/1000/areas)
ax.plot(times,func3(np.array(times)),label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='red')

ax.set_ylim(0,3)
ax.set_xlabel('time (h)')
ax.set_ylabel('density ($mm.mm^{-2}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'density ($mm.mm^{-2}$)')

In [122]:
x,y = times_select,areas_select

In [123]:
coefs = np.polyfit(x,y,2)

In [155]:
0.8*2000/np.log(2)/24

96.17966939259757

In [44]:
from sample.notebooks.analysis.data_info import *

window=800
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "rb"))
column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed","straightness","t","hyph","total_network",'area']
infos = pd.DataFrame(columns=column_names)
inst = (39,269,329)
angles, curvatures, densities,growths,speeds,tortuosities,ts,hyphs,total_lengths = results[inst]
for i,angle in enumerate(angles):
    new_line = pd.DataFrame(
        {   "plate": [plate_number[inst]],
            "inst": [inst],
            "treatment": [treatment],
            "angle": [angle],
            "curvature": [curvatures[i]],
            "density": [densities[i]],
            "growth": [growths[i]],
            "speed": [speeds[i]],
            "straightness": [tortuosities[i]],
             "t": [ts[i]],
             "hyph": [hyphs[i]],
             "total_network" : [total_lengths[i]],
             'area' : [sizes[int(ts[i])]]
        }
    )  # index 0 for
    # mothers need to be modified to resolve multi mother issue
    infos = infos.append(new_line, ignore_index=True)

In [45]:
plate_94 = infos.loc[infos['plate']==94]

In [46]:
plate_94['t'],plate_94['total_network']

(0        0.000000
 1        1.000000
 2        2.000000
 3        3.000000
 4        3.983333
           ...    
 2197    59.016667
 2198    58.016667
 2199    59.016667
 2200    58.016667
 2201    59.016667
 Name: t, Length: 2202, dtype: float64,
 0       1.749240e+05
 1       1.832130e+05
 2       1.908655e+05
 3       1.994984e+05
 4       2.064749e+05
             ...     
 2197    1.187199e+06
 2198    1.160717e+06
 2199    1.187199e+06
 2200    1.160717e+06
 2201    1.187199e+06
 Name: total_network, Length: 2202, dtype: float64)